# Split the clean.csv file into multiple files. Compute VADER sentiment and score

In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange, tqdm_notebook, tqdm

# Define the currency
#CURRENCY = "zilliqa"
#CURRENCY_SYMBOL = "ZIL"
#CURRENCY = "nexo"
#CURRENCY_SYMBOL = "NEXO"
CURRENCY = "bitcoin"
CURRENCY_SYMBOL = "BTC"
tweets_clean_file = f'data/twitter/{CURRENCY_SYMBOL}/{CURRENCY}_tweets_clean.csv'
path = f'data/twitter/{CURRENCY_SYMBOL}' #/{CURRENCY}_tweets_clean.csv'


## Read the cleaned file

In [2]:
df_clean = pd.read_csv("tweets_clean_file")
print(df_clean.shape)
df_clean.head(5)

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(7743302, 7)


,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
0,1375997873305886728,RT : Bitcoin will reach 6 digits in 2021I can ...,Troy Ackerman,50,608,0,Sun Mar 28 02:27:12 +0000 2021
1,1375997869744746496,RT : Bitcoin will not stay below 60k forever. 📈,Robbie,38,101,0,Sun Mar 28 02:27:11 +0000 2021
2,1375997866968195078,RT : I’m giving $100 to one lucky follower tha...,하니 👑6️⃣5️⃣🎹💜,62,1001,0,Sun Mar 28 02:27:10 +0000 2021
3,1375997861997907971,Earn a share of 7 million $REAP cryptocurrency...,misterbhuahlool,41,0,0,Sun Mar 28 02:27:09 +0000 2021
4,1375997858021666816,RT : bitcoin monthly RSI is not even 95. In 20...,Caitlin Long 🔑,97605,1215,0,Sun Mar 28 02:27:08 +0000 2021


In [3]:
a=df_clean.loc[df_clean['UserFollowerCount'] == 'UserFollowerCount']

In [4]:
print(a)

         ID  Text  UserName  UserFollowerCount  RetweetCount  Likes  CreatedAt
1031747  ID  Text  UserName  UserFollowerCount  RetweetCount  Likes  CreatedAt
1868972  ID  Text  UserName  UserFollowerCount  RetweetCount  Likes  CreatedAt
2900720  ID  Text  UserName  UserFollowerCount  RetweetCount  Likes  CreatedAt
4784634  ID  Text  UserName  UserFollowerCount  RetweetCount  Likes  CreatedAt
5816382  ID  Text  UserName  UserFollowerCount  RetweetCount  Likes  CreatedAt


In [5]:
df_clean=df_clean.drop(df_clean.index[[1031747, 1868972, 2900720, 4784634, 5816382]])
#,1868970, 2900714, 4784625, 5816369

In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7743297 entries, 0 to 7743301
Data columns (total 7 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   ID                 object
 1   Text               object
 2   UserName           object
 3   UserFollowerCount  object
 4   RetweetCount       object
 5   Likes              object
 6   CreatedAt          object
dtypes: object(7)
memory usage: 472.6+ MB


In [7]:
df_clean[["UserFollowerCount", "Likes","RetweetCount","ID"]] = df_clean[["UserFollowerCount", "Likes","RetweetCount","ID"]].apply(pd.to_numeric)

In [8]:
df_clean = df_clean.sort_values(by='ID') # the bigger the ID, the most recent the tweet 

In [9]:
df_clean.tail(5)

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
3737947,1386606133167656966,build affiliate business affilateprogram affil...,Brett Murphy,69612,0,0,Mon Apr 26 09:00:38 +0000 2021
3737946,1386606133561872392,RT : 📌 The MoonBud Giveaway | Rules 📌 = - Step...,menang,110,1686,0,Mon Apr 26 09:00:38 +0000 2021
6653608,1386606133561872392,RT : 📌 The MoonBud Giveaway | Rules 📌 = - Step...,menang,110,1686,0,Mon Apr 26 09:00:38 +0000 2021
6653607,1386606137861197827,"👉Follow for more crypto contents, news and giv...",Bitwewe,2,0,0,Mon Apr 26 09:00:39 +0000 2021
3737945,1386606137861197827,"👉Follow for more crypto contents, news and giv...",Bitwewe,2,0,0,Mon Apr 26 09:00:39 +0000 2021


## Sentiment analysis with Vader

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

VADER takes into account 
- negations and contractions (not good, wasn’t good)
- Ponctuation (good!!!), CAPS, emotes :), emojis 
- Intensificators (very, kind of), acronyms ‘lol’
- Scores between -1.0 (negative) and 1.0 (positive)

We will use this sentiment analysis of the tweets to calculate a score that will represent the importance of each tweet.

In [10]:
analyzer = SentimentIntensityAnalyzer()
compound = []
for i,s in enumerate(tqdm(df_clean['Text'])):
    vs = analyzer.polarity_scores(s)
    compound.append(vs["compound"])
df_clean["compound"] = compound
df_clean.head(2)

100%|██████████| 7743297/7743297 [21:10<00:00, 6093.53it/s] 


,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt,compound
5053236,1375226282036764674,RT : This is what happens to Bitcoin when opti...,Ronald,21,1103,0,Thu Mar 25 23:21:10 +0000 2021,-0.296
2137574,1375226282036764674,RT : This is what happens to Bitcoin when opti...,Ronald,21,1103,0,Thu Mar 25 23:21:10 +0000 2021,-0.296


In [11]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7743297 entries, 5053236 to 3737945
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 int64  
 1   Text               object 
 2   UserName           object 
 3   UserFollowerCount  int64  
 4   RetweetCount       int64  
 5   Likes              int64  
 6   CreatedAt          object 
 7   compound           float64
dtypes: float64(1), int64(4), object(3)
memory usage: 531.7+ MB


## Calculate a score for each tweet

To calculate the score for each tweet, we use different variables to which we had a weight based on its importance.

The compound column represents the sentiment of the tweets and its value is between -1 and 1.

We also use the number of retweets, the number of likes, and the number of users that follow the tweet's author.

In [12]:
df_clean[["UserFollowerCount", "Likes","RetweetCount"]] = df_clean[["UserFollowerCount", "Likes","RetweetCount"]].apply(pd.to_numeric)
scores = []
for i, s in tqdm(df_clean.iterrows(), total=df_clean.shape[0]):
    scores.append(s["compound"] * (s["UserFollowerCount"]+1) * (s["Likes"]+1))
df_clean["score"] = scores
df_clean.head(2)

100%|██████████| 7743297/7743297 [20:22<00:00, 6332.13it/s]


,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt,compound,score
5053236,1375226282036764674,RT : This is what happens to Bitcoin when opti...,Ronald,21,1103,0,Thu Mar 25 23:21:10 +0000 2021,-0.296,-6.512
2137574,1375226282036764674,RT : This is what happens to Bitcoin when opti...,Ronald,21,1103,0,Thu Mar 25 23:21:10 +0000 2021,-0.296,-6.512


In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7743297 entries, 5053236 to 3737945
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 int64  
 1   Text               object 
 2   UserName           object 
 3   UserFollowerCount  int64  
 4   RetweetCount       int64  
 5   Likes              int64  
 6   CreatedAt          object 
 7   compound           float64
 8   score              float64
dtypes: float64(2), int64(4), object(3)
memory usage: 590.8+ MB


## Split dataframe and save it into multiple files

In [14]:
from datetime import datetime

In [15]:
n = 10000  #chunk row size
chunks_df = [df_clean[i:i+n] for i in range(0,df_clean.shape[0],n)]

sep_char = '~'
for chunk_df in chunks_df:
    chunk_min = chunk_df['ID'].min()
    chunk_max = chunk_df['ID'].max()
    date_from = (datetime.strptime(chunk_df.iloc[0]['CreatedAt'], '%a %b %d %X %z %Y')).strftime('%Y-%m-%d %H-%M-%S')
    date_to = (datetime.strptime(chunk_df.iloc[-1]['CreatedAt'], '%a %b %d %X %z %Y')).strftime('%Y-%m-%d %H-%M-%S')
    print(date_from, date_to)

    # Write into csv
    chunk_df.to_csv(f"{path}/{date_from}{sep_char}{date_to}.csv", header=True, index=False)

2021-03-25 23-21-10 2021-03-26 00-01-53
2021-03-26 00-01-53 2021-03-26 00-33-33
2021-03-26 00-33-33 2021-03-26 01-15-32
2021-03-26 01-15-32 2021-03-26 02-00-13
2021-03-26 02-00-13 2021-03-26 02-51-26
2021-03-26 02-51-26 2021-03-26 03-37-00
2021-03-26 03-37-03 2021-03-26 04-21-18
2021-03-26 04-21-18 2021-03-26 05-14-33
2021-03-26 05-14-33 2021-03-26 06-06-40
2021-03-26 06-06-40 2021-03-26 06-57-16
2021-03-26 06-57-16 2021-03-26 07-44-20
2021-03-26 07-44-20 2021-03-26 08-26-58
2021-03-26 08-26-58 2021-03-26 09-03-40
2021-03-26 09-03-40 2021-03-26 09-43-50
2021-03-26 09-43-50 2021-03-26 10-23-00
2021-03-26 10-23-02 2021-03-26 11-01-37
2021-03-26 11-01-37 2021-03-26 11-31-34
2021-03-26 11-31-34 2021-03-26 12-01-42
2021-03-26 12-01-42 2021-03-26 12-29-04
2021-03-26 12-29-04 2021-03-26 12-49-51
2021-03-26 12-49-51 2021-03-26 13-06-13
2021-03-26 13-06-13 2021-03-26 13-23-15
2021-03-26 13-23-15 2021-03-26 13-43-56
2021-03-26 13-43-56 2021-03-26 14-05-41
2021-03-26 14-05-41 2021-03-26 14-26-14


2021-03-31 01-09-46 2021-03-31 01-38-01
2021-03-31 01-38-01 2021-03-31 02-06-11
2021-03-31 02-06-11 2021-03-31 02-34-04
2021-03-31 02-34-04 2021-03-31 03-01-52
2021-03-31 03-01-53 2021-03-31 03-29-45
2021-03-31 03-29-45 2021-03-31 04-01-25
2021-03-31 04-01-25 2021-03-31 04-28-24
2021-03-31 04-28-24 2021-03-31 04-51-29
2021-03-31 04-51-29 2021-03-31 05-17-30
2021-03-31 05-17-30 2021-03-31 05-42-59
2021-03-31 05-42-59 2021-03-31 06-08-08
2021-03-31 06-08-08 2021-03-31 06-31-59
2021-03-31 06-31-59 2021-03-31 06-53-16
2021-03-31 06-53-16 2021-03-31 07-16-47
2021-03-31 07-16-47 2021-03-31 07-43-22
2021-03-31 07-43-22 2021-03-31 08-02-04
2021-03-31 08-02-06 2021-03-31 08-21-33
2021-03-31 08-21-33 2021-03-31 08-44-10
2021-03-31 08-44-10 2021-03-31 09-08-10
2021-03-31 09-08-11 2021-03-31 09-29-58
2021-03-31 09-29-58 2021-03-31 09-52-31
2021-03-31 09-52-31 2021-03-31 10-12-56
2021-03-31 10-12-56 2021-03-31 10-33-47
2021-03-31 10-33-47 2021-03-31 10-55-44
2021-03-31 10-55-44 2021-03-31 11-08-34


2021-04-03 16-48-35 2021-04-03 17-12-13
2021-04-03 17-12-13 2021-04-03 17-35-01
2021-04-03 17-35-01 2021-04-03 17-57-39
2021-04-03 17-57-41 2021-04-03 18-19-48
2021-04-03 18-19-48 2021-04-03 18-39-21
2021-04-03 18-39-21 2021-04-03 19-03-48
2021-04-03 19-03-48 2021-04-03 19-28-38
2021-04-03 19-28-38 2021-04-03 19-56-15
2021-04-03 19-56-15 2021-04-03 20-25-32
2021-04-03 20-25-32 2021-04-03 20-50-46
2021-04-03 20-50-46 2021-04-03 21-17-26
2021-04-03 21-17-26 2021-04-03 21-44-16
2021-04-03 21-44-17 2021-04-03 22-31-27
2021-04-03 22-31-27 2021-04-03 23-19-20
2021-04-03 23-19-20 2021-04-04 00-13-32
2021-04-04 00-13-33 2021-04-04 01-04-18
2021-04-04 01-04-18 2021-04-04 01-51-54
2021-04-04 01-51-54 2021-04-04 02-45-06
2021-04-04 02-45-06 2021-04-04 03-47-39
2021-04-04 03-47-39 2021-04-04 04-30-23
2021-04-04 04-30-23 2021-04-04 05-04-01
2021-04-04 05-04-01 2021-04-04 05-54-25
2021-04-04 05-54-25 2021-04-04 06-46-09
2021-04-04 06-46-09 2021-04-04 07-18-31
2021-04-04 07-18-31 2021-04-04 08-03-43


2021-04-20 14-08-36 2021-04-20 14-56-22
2021-04-20 14-56-22 2021-04-20 15-41-06
2021-04-20 15-41-06 2021-04-20 16-30-38
2021-04-20 16-30-38 2021-04-20 17-22-23
2021-04-20 17-22-23 2021-04-20 18-16-25
2021-04-20 18-16-25 2021-04-20 19-13-53
2021-04-20 19-13-53 2021-04-20 20-11-31
2021-04-20 20-11-31 2021-04-20 21-06-57
2021-04-20 21-06-57 2021-04-20 22-10-59
2021-04-20 22-10-59 2021-04-20 23-31-24
2021-04-20 23-31-24 2021-04-21 00-48-08
2021-04-21 00-48-08 2021-04-21 01-50-38
2021-04-21 01-50-38 2021-04-21 03-10-38
2021-04-21 03-10-38 2021-04-21 04-34-16
2021-04-21 04-34-16 2021-04-21 05-51-28
2021-04-21 05-51-28 2021-04-21 07-09-12
2021-04-21 07-09-12 2021-04-21 08-11-54
2021-04-21 08-11-54 2021-04-21 09-24-27
2021-04-21 09-24-27 2021-04-21 10-34-07
2021-04-21 10-34-07 2021-04-21 11-33-42
2021-04-21 11-33-42 2021-04-21 12-27-39
2021-04-21 12-27-39 2021-04-21 13-26-18
2021-04-21 13-26-18 2021-04-21 14-20-37
2021-04-21 14-20-37 2021-04-21 15-13-43
2021-04-21 15-13-43 2021-04-21 15-48-29


## Update var.csv

In [16]:
import glob
import numpy as np

ENVS = ['CRYPTO', 'LINE_COUNT', 'MOST_RECENT_FILE', 'MOST_RECENT_ID'] # Stored in var.csv

def get_var(key, crypto):
    df_var = pd.read_csv("data/twitter/var_twitter.csv", sep=',',dtype={'LINE_COUNT': np.int32})
    return df_var[key].loc[df_var['CRYPTO'] == crypto].values[0]

def update_var(key, value, crypto):
    df_var = pd.read_csv("data/twitter/var_twitter.csv", sep=',',dtype={'LINE_COUNT': np.int32})
    df_var[key].loc[df_var['CRYPTO'] == crypto] = str(value)
    df_var.to_csv("data/twitter/var_twitter.csv", index=False)
    
def add_new_crypto(crypto):
    df_var = pd.read_csv("data/twitter/var_twitter.csv", sep=',',dtype={'LINE_COUNT': np.int32})
    if df_var[ENVS[0]].loc[df_var['CRYPTO'] == crypto].empty:
        new_line = pd.DataFrame([[crypto,-1,"",0]], columns=ENVS)
        df_var = df_var.append(new_line)
        df_var.to_csv("data/twitter/var_twitter.csv", index=False)

In [17]:
files = glob.glob(f"{path}/*~*.csv")
files = sorted(files)
last_file = files[-1]
#print(last_file)
last_df = pd.read_csv(last_file)
last_elem = last_df.tail(1)
print(last_df.iloc[0]['ID'])
#print(last_df.shape[0])

add_new_crypto(CURRENCY_SYMBOL)
update_var(ENVS[1], last_df.shape[0], CURRENCY_SYMBOL)
update_var(ENVS[2], last_file, CURRENCY_SYMBOL)
update_var(ENVS[3], last_df.iloc[0]['ID'], CURRENCY_SYMBOL)

1386600586196525059


/home/nibbler/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [18]:
var_df = pd.read_csv('data/twitter/var_twitter.csv')
var_df

,CRYPTO,LINE_COUNT,MOST_RECENT_FILE,MOST_RECENT_ID
0,BTC,3284,data/twitter/BTC/2021-04-26 08-38-36~2021-04-2...,1386600586196525059
